In [2]:
# Code for extracting prediction articles
# Logic (full_articles.json - articles_used_for_training)
# Author: Pandramishi Naga Sirisha
# Date: 26th May 2020
## Enhancements:
#1. encapsulate in a function, modify week2 code as well so you can import that function
#2. write code to delete _to_convert.csv file

In [1]:
import pandas as pd
import urllib.request
import json 
import json
import datetime
import pytz
import dateutil.parser
from collections import defaultdict
import random
import json
import csv

In [2]:
def json_to_csv(filename, list_name, file_suffix):
    """creates a csv containing title and description of articles in a given JSON.
    Re-using this function from Sampling_articles_by_month.ipynb from week_2"""
    ind = 0
    title_list = []
    desc_list = []
    date_list =[]
    source_list = []
    invalid_articles = 0
    
    for json_dict in list_name:
        #check if valid title exists
        if json_dict["title"]:
            title_list.append(json_dict["title"])
            desc_list.append(json_dict["description"])
            date_list.append(dateutil.parser.parse(json_dict['publishedAt'][1]).date())
            source_list.append("CBC")
            ind += 1
            
        else:
            invalid_articles += 1
            
    print("Invalid articles :", invalid_articles)        
    prefix = filename.split(".")[0]
    out_filename = prefix  + "_" + str(ind) + "_" + file_suffix + ".csv"
    with open(out_filename, 'w') as myfile:
        writer = csv.writer(myfile)
        #wr.writerow(mylist)
        writer.writerow(["title", "description"])
        for i in range(len(title_list)):
            writer.writerow([title_list[i], desc_list[i], date_list[i], source_list[i]])
    myfile.close()
    print("Output file name is :", out_filename)
    return out_filename
                
# json_to_csv("mortgage_rates_CBC_article.json")

In [3]:
data_folder = "./Full_articles_unannotated/"

In [10]:
file_name = "TSX_CBC_article.json"
economic_indicator_name = "stock_market"

In [5]:
with open(data_folder + file_name ) as f:
    list_of_json = json.load(f)

In [6]:
# Processing full articles' dataframe to get required format
# Read article in json format and output as csv to process data easily as dataframes
full_article_file_name = json_to_csv(file_name, list_of_json, "convert")
full_df = pd.read_csv(full_article_file_name, sep=',', names=['title', 'description', 'publishedAt', 'source'], skiprows = 1)
full_df["title_desc"] = full_df ['title'] + ". " + full_df['description']
full_df = full_df.drop(columns = ['title', 'description'])
full_df = full_df[['source','title_desc', 'publishedAt' ]]
full_df = full_df.dropna(how='any',axis=0)
full_df.head()

Invalid articles : 84
Output file name is : TSX_CBC_article_886_convert.csv


,source,title_desc,publishedAt
0,CBC,TSX and NYSE closed for Good Friday but Asian ...,2020-04-10
1,CBC,TSX and Dow make gains Thursday but coronaviru...,2020-03-19
3,CBC,"TSX and Dow plunge back into the red Friday, c...",2020-03-20
5,CBC,TSX and Dow Jones bounce back somewhat after m...,2020-03-17
6,CBC,TSX and Dow Jones lose another 10% as coronavi...,2020-03-16


In [7]:
# Convert dataframe type to datetime
import pandas
full_df['publishedAt'] =  pd.to_datetime(full_df['publishedAt'], format='%Y-%m-%d')
full_df.shape

(423, 3)

In [8]:
#Code to check if all the articles are from 2019-05-12 to 2020-05-12
import pandas
cond1 = full_df.publishedAt > pd.Timestamp(2019,5,12)
x = full_df.loc[cond1, ]
x.shape

(60, 3)

In [11]:
# Processing training dataframe to get required format
training_df = pd.read_csv("./final_cbc_annotations/" + economic_indicator_name+ "_combined_annotations_CBC.csv", names = ['title','description', 'publishedAt', 'title_desc_sent_1', 'source'], skiprows = 1)
training_df["title_desc"] = training_df ['title'] + ". " + training_df['description']
training_df = training_df.drop(columns = ['title', 'description','title_desc_sent_1'])
training_df = training_df[['source','title_desc', 'publishedAt' ]]
training_df.iloc[0]['title_desc']

"HBC shares up on news of likely takeover deal. Baker group poised to gain control of retailer after agreeing to Catalyst Capital's higher asking price"

In [12]:
# Do a left join  between two dataframes
final = pd.merge(full_df,training_df,on='title_desc',how='left', indicator=True)
# final = final.dropna(how='any',axis=0)

# Getting only (left_only) rows to make it left OUTER join
final = final[final['_merge'] == 'left_only']
final = final.drop(columns = ['source_y','publishedAt_y','_merge'])
final.columns = ['source','title_desc', 'publishedAt']
final.shape
# final.ix[62]['title_desc'] == training_df.iloc[0]['title_desc']

(421, 3)

In [13]:
## Output final prediction file
final.to_csv(file_name.split(".")[0] + "_to_predict.csv",sep=',')